# Exercise 5

In [1]:
!pip install streamlit

In [2]:
## !streamlit hello

In [3]:
!pip install langchain

In [4]:
!pip install langdetect

In [5]:
!pip install -U torch

## Set Enviroment 

In [6]:
import os
import pandas as pd

In [7]:
import langdetect
from langdetect import DetectorFactory, detect, detect_langs

In [8]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [9]:

from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [10]:
# Embedding facilities
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
# Pipelines
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

## Load Documents

In [12]:
import myloadlib
from myloadlib import loadDir, loadFile, loadWiki, loadYoutube, readAPI

In [13]:
import importlib 
importlib.reload(myloadlib)

<module 'myloadlib' from '/Users/mariamcnally/Desktop/SW2Semester/AI/OLA5/myloadlib.py'>

In [14]:
# Collect all here
documents = []

## Load single files

In [15]:
file = "/Users/mariamcnally/Desktop/SW2Semester/AI/DataWrangling.pdf"

In [16]:
!pip install pypdf

In [17]:
docs = myloadlib.loadFile(file)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)


In [18]:
documents.extend(docs)
len(documents)

2

In [19]:
# metadata of loaded Document
docs[0].metadata 

{'source': '/Users/mariamcnally/Desktop/SW2Semester/AI/DataWrangling.pdf',
 'page': 0}

Content of page [0] = page 1. 

In [20]:
documents[0].page_content
#docs[0].page_content[:1000]
# First 1000 charactors.

'Data Wrangling To get the data into a good state before analysing it there are some steps we need to go  through . The following process is a basic outline of what needs to happen before data is  ready for modelling and analysis. • Data\tDiscovery:\tThis\tis\tan\tall-encompassing\tterm\tthat\tdescribes\tunderstanding\twhat\tyour\tdata\tis\tall\tabout.\tIn\tthis\tfirst\tstep,\tyou\tget\tfamiliar\twith\tyour\tdata\t\tand\tensure\tit\tis\tcapable\tof\tbeing\tused\tto\thelp\twork\ton\tyour\tproblem.\t\t• Data\tStructuring:\tWhen\tyou\tcollect\traw\tdata,\tit\tinitially\tis\tin\tall\tshapes\tand\t\tsizes,\tand\thas\tno\tdefinite\tstructure.\tSuch\tdata\tneeds\tto\tbe\trestructured\tto\tsuit\tthe\t\tanalytical\tmodel\tthat\tyour\tenterprise\tplans\tto\tdeploy\t\t\t• Data\tCleaning:\tRaw\tdata\tcomes\twith\tsome\terrors\tthat\tneed\tto\tbe\tfixed\tbefore\t\tdata\tis\tpassed\ton\tto\tthe\tnext\tstage.\tCleaning\tinvolves\tthe\ttackling\tof\toutliers,\t\tmaking\tcorrections,\tor\tdeleting\tbad

## Load wiki

In [21]:
subject = "Testing wiki load"

In [22]:
lang = 'en'

In [23]:
!pip install wikipedia

In [24]:
docs = myloadlib.loadWiki(subject, lang, 2)

In [25]:
documents.extend(docs)

Should be 4 at the moment, but will update everytime its run, and or other Docs/documents are ran again

In [26]:
len(documents)

4

In [27]:
## Choose not to show Load youtube

In [28]:
!pip install spacy
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.0.0/en_core_web_md-3.0.0.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 MB 2.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.2/989.2 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [648 lines of output]
        Using cached setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
        Using cached cymem-2.0.8-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.4 kB)
        Using cached preshed-3.0.9-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.2 kB)
        Using cached murmurhash-1.0.10-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.0 kB)
           ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/189.7 kB ? eta -:--:--
           ━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/189.7 kB 1.6 MB/s eta 0:00:01
           ━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 102.4/189.7 kB 1.5 MB/

In [29]:
!pip install wordcloud

In [30]:
import myutils
from myutils import chunkDocs, langDetect, wordCloud

In [31]:
import importlib 
importlib.reload(myutils)

<module 'myutils' from '/Users/mariamcnally/Desktop/SW2Semester/AI/OLA5/myutils.py'>

In [32]:
splits = myutils.chunkDocs(documents, 350)  
splits

[Document(page_content='Data Wrangling To get the data into a good state before analysing it there are some steps we need to go  through . The following process is a basic outline of what needs to happen before data is  ready for modelling and analysis. •', metadata={'source': '/Users/mariamcnally/Desktop/SW2Semester/AI/DataWrangling.pdf', 'page': 0}),
 Document(page_content='Data\tDiscovery:\tThis\tis\tan\tall-encompassing\tterm\tthat\tdescribes\tunderstanding\twhat\tyour\tdata\tis\tall\tabout.\tIn\tthis\tfirst\tstep,\tyou\tget\tfamiliar\twith\tyour\tdata\t\tand\tensure\tit\tis\tcapable\tof\tbeing\tused\tto\thelp\twork\ton\tyour\tproblem.\t\t•', metadata={'source': '/Users/mariamcnally/Desktop/SW2Semester/AI/DataWrangling.pdf', 'page': 0}),
 Document(page_content='Data\tStructuring:\tWhen\tyou\tcollect\traw\tdata,\tit\tinitially\tis\tin\tall\tshapes\tand\t\tsizes,\tand\thas\tno\tdefinite\tstructure.\tSuch\tdata\tneeds\tto\tbe\trestructured\tto\tsuit\tthe\t\tanalytical\tmodel\tthat\tyo

In [33]:
len(splits)

50

In [34]:
splits[7]

Document(page_content='credible models as everything else depends on the quality of the data.  Examples\tof\tthe\tsteps', metadata={'source': '/Users/mariamcnally/Desktop/SW2Semester/AI/DataWrangling.pdf', 'page': 0})

In [35]:
# put splits in dataframe
df = pd.DataFrame(splits, columns=['page_content', 'metadata', 'type'])
df.sample(3)

,page_content,metadata,type
25,"(page_content, Equipment Regulations 1998 UK l...","(metadata, {'title': 'Load testing', 'summary'...","(type, Document)"
34,"(page_content, prolonged viral shedding and ha...","(metadata, {'title': 'Viral load', 'summary': ...","(type, Document)"
41,"(page_content, A 2010 review study by Puren et...","(metadata, {'title': 'Viral load', 'summary': ...","(type, Document)"


In [36]:
df['page_content'][0]

('page_content',
 'Data Wrangling To get the data into a good state before analysing it there are some steps we need to go  through . The following process is a basic outline of what needs to happen before data is  ready for modelling and analysis. •')

In [37]:
df['metadata'][0]

('metadata',
 {'source': '/Users/mariamcnally/Desktop/SW2Semester/AI/DataWrangling.pdf',
  'page': 0})

In [38]:
im, longstring = myutils.wordCloud(df, 'page_content')

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a Python package or a valid path to a data directory.